In [34]:
import pandas as pd

# Load dataset
file_2022 = "BITRE-Road-Vehicles-Australia-January-2022-Re-Issue.xlsx"
file_2024 = "bitre-road-vehicles-australia--january2024.xlsx"

df_2022 = pd.read_excel(file_2022, sheet_name="Table 1", skiprows=6)
df_2024 = pd.read_excel(file_2024, sheet_name="Table 1", skiprows=6)

In [102]:
# Extract vehicle data only for victoria data
def extract_vehicle_data(df):
    records = []
    current_vehicle_type = None

    for _, row in df.iterrows():
        year = row["Unnamed: 1"]
        vic_value = row["Unnamed: 3"]
        vehicle_type_candidate = row["Unnamed: 2"]

        # Save vehicle type label from top row
        if pd.isna(year) and pd.notna(vehicle_type_candidate):
            current_vehicle_type = vehicle_type_candidate

        # Get data if there is a year and vehicle value
        elif pd.notna(year) and current_vehicle_type is not None:
            try:
                year = int(year)
                vic_value = float(vic_value)
                records.append({
                    "year": year,
                    "vehicle_type": current_vehicle_type,
                    "value": vic_value
                })
            except:
                continue
    return pd.DataFrame(records)

In [104]:
# Extract data from both dataset
df_vic_2022 = extract_vehicle_data(df_2022) # data from 2021-2022
df_vic_2024 = extract_vehicle_data(df_2024) # data from 2022-2024

# Combine the data and remove duplicate
df_combined = pd.concat([df_vic_2022, df_vic_2024], ignore_index=True)
df_combined = df_combined.drop_duplicates().sort_values(by=["vehicle_type", "year"])

In [106]:
df_combined.head(100)

,year,vehicle_type,value
10,2021,Articulated trucks,31343.0
11,2022,Articulated trucks,32635.0
36,2023,Articulated trucks,33793.0
37,2024,Articulated trucks,34927.0
14,2021,Buses,20053.0
15,2022,Buses,20145.0
2,2021,Campervans,21449.0
3,2022,Campervans,22779.0
24,2023,Campervans,24222.0
25,2024,Campervans,24805.0


In [108]:
df_combined["vehicle_type"].unique()

array(['Articulated trucks', 'Buses', 'Campervans', 'Heavy buses',
       'Heavy rigid trucks', 'Light buses', 'Light commercial vehicles',
       'Light rigid trucks', 'Motorcycles',
       'Non-freight-carrying vehicles', 'Passenger vehicles',
       'Total motor vehicles'], dtype=object)

In [110]:
# Categorized the vehicle based on the type
def categorize_vtype(vtype):
    vtype = vtype.strip().lower()
    if vtype == "passenger vehicles":
        return "Passenger vehicles"
    elif vtype == "light commercial vehicles":
        return "Light commercial vehicles"
    elif vtype == "motorcycles":
        return "Motorcycles"
    elif vtype == "total motor vehicles":
        return "All types"
    else:
        return "Others"

# Apply the fucntion
df_combined["vehicle_group"] = df_combined["vehicle_type"].apply(categorize_vtype)
df_combined.head(10)

,year,vehicle_type,value,vehicle_group
10,2021,Articulated trucks,31343.0,Others
11,2022,Articulated trucks,32635.0,Others
36,2023,Articulated trucks,33793.0,Others
37,2024,Articulated trucks,34927.0,Others
14,2021,Buses,20053.0,Others
15,2022,Buses,20145.0,Others
2,2021,Campervans,21449.0,Others
3,2022,Campervans,22779.0,Others
24,2023,Campervans,24222.0,Others
25,2024,Campervans,24805.0,Others


In [114]:
# change vehicle into int
df_combined["value"] = df_combined['value'].astype(int)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 10 to 52
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           44 non-null     int64 
 1   vehicle_type   44 non-null     object
 2   value          44 non-null     int64 
 3   vehicle_group  44 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.7+ KB


In [116]:
# save combined files into csv
df_combined.to_csv("vehicle_ownership.csv", index=False)